In [ ]:
import itertools
import pprint
import re
import pickle
import os
import sys
import requests
import json
import bs4 as bs
import h5py as h5

### Read Resource TSV

In [ ]:
# Read Resource TSV
open_resource = open("resource.tsv", "r")
header = open_resource.readline()
example = open_resource.readline()

# Key = ID, Value = Resource Name
resource_dict = dict()
for row in open_resource:
    split_row =  row.split("\t")
    
    if '\n' == split_row[len(split_row)-1]:
        split_row[len(split_row)-1] = "NULL"
        
    resource_id = split_row[2]
    resource_name = split_row[0]
    
    count = 2
    field_list = list()
    while 1 < count < 11:
        if split_row[count] != '':
            field_value = split_row[count]
            field_value = field_value.rstrip()
            if '"' in field_value:
                field_value = field_value.replace('"', '')
            field_list.append(field_value)
        else:
            field_list.append("NULL")
        count += 1
    
    resource_dict[resource_id] = field_list

pprint.pprint(resource_dict)

### Read Schema

In [ ]:
f = open("schema.txt", "r")

schema_dict = dict()
name = ''
table_dict = dict()
count = False

for row in f:
    row = row.rstrip()
    if "---" == row:
        count = True
        continue
    
    if count == True:
        row_list = row.split("`")
        name = row_list[1]
        schema_dict[name] = list()
        schema_dict[name].append(row)
        
        table_dict[name] = list()
        count = False
    
    else:
        schema_dict[name].append(row)
        if row[:3] == ' `':
            row = row.split('`')[1]
            row = {row:list()}
            table_dict[name].append(row)

f.close()

### Show Schema Keys

In [ ]:
pprint.pprint(schema_dict.keys())

In [ ]:
def printSchema(table_name, open_sql):
    if table_name == "begin":
        count = 0
        for row in schema_dict[table_name]:
            if count == 0:
                count += 1
                continue
            else:
                open_sql.write("\n{}".format(str(row)))
    else:
        for row in schema_dict[table_name]:
            open_sql.write("\n{}".format(str(row)))

### Write beginning

In [ ]:
# Write to master_sql
open_sql = open("master.sql", "w")
printSchema('begin', open_sql)

# ----------------------------------------------------------

### Write Resource Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('resource', open_sql)

### Insert values Resource into master SQL

In [ ]:
def beginInsert(table_name):
    open_sql.write('\n\nLOCK TABLES `{}` WRITE;'.format(table_name))
    open_sql.write('\nINSERT INTO `{}` VALUES'.format(table_name))

In [ ]:
beginInsert('resource')

In [ ]:
def fillTable(resource_dict, open_sql):
    index = 0
    for key, value in resource_dict.items():
        if index == len(resource_dict.keys())-1:
            open_sql.write('\n(')
            for count, entry in enumerate(value):
                entry = str(entry)
                if count == len(value)-1:
                    entry = entry.rstrip()
                    if entry == " " or entry == "":
                        open_sql.write("{});".format("NULL"))
                    elif entry.isnumeric():
                        open_sql.write("{});".format(entry))
                    elif "NULL" == entry or "Null" == entry:
                        open_sql.write("{});".format(entry))
                    else:
                        open_sql.write('"{}");'.format(entry))
                else:
                    if entry == " " or entry == "":
                        open_sql.write("{},".format("NULL"))
                    elif entry.isnumeric():
                        open_sql.write("{},".format(entry))
                    elif "NULL" == entry:
                        open_sql.write("{},".format(entry))
                    else:
                        open_sql.write('"{}",'.format(entry))
            break
            
        open_sql.write('\n(')   
        for count, entry in enumerate(value):
            entry = str(entry)
            if count == len(value)-1:
                entry = entry.rstrip()
                if entry == " " or entry == "":
                    open_sql.write("{}),".format("NULL"))
                elif entry.isnumeric():
                    open_sql.write("{}),".format(entry))
                elif "NULL" == entry or "Null" == entry:
                    open_sql.write("{}),".format(entry))
                else:
                    open_sql.write('"{}"),'.format(entry))
            else:
                if entry == " " or entry == "":
                    open_sql.write("{},".format("NULL"))
                elif entry.isnumeric():
                    open_sql.write("{},".format(entry))
                elif "NULL" == entry:
                    open_sql.write("{},".format(entry))
                else:
                    open_sql.write('"{}",'.format(entry))
        index += 1
        
    open_sql.write("\nUNLOCK TABLES;")

In [ ]:
fillTable(resource_dict, open_sql)

# ----------------------------------------------------------

### Read Publications TSV

In [ ]:
# Read Publication TSV
open_publication = open("publications.tsv", "r")

# Read Header
header = open_publication.readline()
# Read Example
example = open_publication.readline()

# Key = ID, Value = Publications Name
publication_dict = dict()
for row in open_publication:
    split_row =  row.split("\t")
    
    if split_row[1] == '':
        continue
    
    if '\n' in split_row[len(split_row)-1]:
        split_row[len(split_row)-1] = split_row[len(split_row)-1].rstrip()
        
    publication_id = split_row[0]
    
    count = 0
    field_list = list()
    while count < 17:
        if split_row[count] != '':
            field_value = split_row[count]
            if '"' in field_value:
                field_value = field_value.replace('"', '')
            field_list.append(field_value)
        else:
            field_list.append("Null")
        count += 1
    
    publication_dict[publication_id] = field_list

open_publication.close()
#pprint.pprint(publication_dict)

### Write Publication Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('publication', open_sql)

### Insert values Publication into master SQL

In [ ]:
beginInsert('publication')

In [ ]:
fillTable(publication_dict, open_sql)

# ----------------------------------------------------------

### Read Dataset TSV

In [ ]:
# Read Dataset TSV
open_dataset = open("dataset.tsv", "r")

# Read Header
header = open_dataset.readline()
# Read Example
example = open_dataset.readline()

# Key = ID, Value = Publications Name
dataset_dict = dict()
for row in open_dataset:
    split_row =  row.split("\t")
    
    if split_row[1] == '':
        continue
    
    if '\n' in split_row[len(split_row)-1]:
        split_row[len(split_row)-1] = split_row[len(split_row)-1].rstrip()
        
    dataset_id = split_row[0]
    
    count = 0
    field_list = list()
    while count < 24:
        if split_row[count] != '':
            field_value = split_row[count]
            if '"' in field_value:
                field_value = field_value.replace('"', '')
            field_list.append(field_value)
        else:
            field_list.append("NULL")
        count += 1
    
    dataset_dict[dataset_id] = field_list

open_dataset.close()
pprint.pprint(dataset_dict)

### Write Dataset Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('dataset', open_sql)

### Insert values Dataset into master SQL

In [ ]:
beginInsert('dataset')

In [ ]:
fillTable(dataset_dict, open_sql)

# ----------------------------------------------------------

### Read Gene List TSV

In [ ]:
# Read Gene List TSV
open_genelist = open("gene_list.tsv", "r")

header = open_genelist.readline()

gene_symbol_list = list()
for row in open_genelist:
    split_genelist = row.split("\t")
    gene_symbol = split_genelist[0]
    gene_id = split_genelist[1]

    gene_symbol_list.append(gene_symbol)

open_genelist.close()

def extractHarCom(begin, end, compare_list, field_num):
    # Create Gene Dictionary
    gen_dict = dict() # Key: ID, Value: List of field
    
    # Extract information from Harmonizome.sql
    begin = begin
    end = end + 1

    length = end - begin
    id_index = 0
    with open('harmonizome.sql', 'r') as f:
        count = 0
        for line in itertools.islice(f, begin-1, end):
            if count == length:
                print(line)
                break
            count += 1
            quote = False
            slash = False

            row = line.rstrip()
            if row[0] == '(':
                final_str = ''
                for i in row:
                    if i == "'" and quote == False:
                        quote = True
                    elif slash == True:
                        slash = False
                    elif "\\" == i and quote == True:
                        slash = True
                    elif i == "'" and quote == True:
                        quote = False
                    elif quote == True:
                        if "," in i:
                            final_str += '*'
                        else:
                            final_str += i
                    else:
                        final_str += i
                final_str = final_str.replace("'", "")
                final_str = final_str[1:-2]
                split_list = final_str.split(',')
                gene = split_list[field_num]

                if gene in compare_list:
                    row_list = list()

                    for value in split_list:
                        if '*' in value:
                            row_list.append(value.replace("*", ","))
                        else:
                            row_list.append(value)

                    gen_dict[gene] = row_list

                id_index = split_list[0]
    return(gen_dict, id_index)

gene_dict, id_index = extractHarCom(71906861, 71963595, gene_symbol_list, 1)
pprint.pprint(gene_dict)

### Write Gene List Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('gene', open_sql)

### Insert values Gene List into master SQL

In [ ]:
beginInsert('gene')

### Create output directory
Create an output directory if it does not exit already and change current directory to output directory

In [ ]:
def TableOutput(table_name, gen_dict, schema_list):
    start_dir = os.getcwd()
    output_path = '{}/achilles_tables'.format(start_dir)
    field = ''
    if os.path.isdir(output_path):
        # Change the current working directory to output directory
        os.chdir(output_path)
        write_table = open('{}.tsv'.format(table_name), "w")
        for count, row in enumerate(schema_list): 
            x = re.search("  `(.*?)`", row)
            
            if not x and field != '':
                write_table.write("{}".format(field)) 
                field = ''
            elif x and field != '':
                write_table.write("{}\t".format(field)) 
            
            if x:
                start_end_tup = x.span()
                start_index = int(start_end_tup[0]) + 3
                end_index = int(start_end_tup[1]) - 1
                field = row[start_index:end_index]    

        write_table.write("\n")

        for key, value_list in gen_dict.items():
            for count, value in enumerate(value_list):
                if count == len(value_list)-1:
                    write_table.write("{}".format(value)) 
                else:
                    write_table.write("{}\t".format(value)) 
            write_table.write("\n")
    else:
        field = ''
        # Create output directory if it does not exist
        os.mkdir(output_path)
        os.chdir(output_path)
        write_table = open('{}.tsv'.format(table_name), "w")
        for count, row in enumerate(schema_list):   
            x = re.search("  `(.*?)`", row)
            
            x = re.search("  `(.*?)`", row)
            
            if not x and field != '':
                write_table.write("{}".format(field)) 
                field = ''
            elif x and field != '':
                write_table.write("{}\t".format(field)) 
            
            if x:
                start_end_tup = x.span()
                start_index = int(start_end_tup[0]) + 3
                end_index = int(start_end_tup[1]) - 1
                field = row[start_index:end_index]   
                
        write_table.write("\n")

        for key, value_list in gen_dict.items():
            for count, value in enumerate(value_list):
                if count == len(value_list)-1:
                    write_table.write("{}".format(value)) 
                else:
                    write_table.write("{}\t".format(value)) 
            write_table.write("\n")
    
    write_table.close()    
    os.chdir(start_dir)

### Get entrez id and description from gene symbol

In [ ]:
def get_url(url):
    myResponse = requests.get(url)
    entrez_id = ""
    uniprot_ids = ""
    gene_group = ""
    gene_group_id = ""
    
    if (myResponse.ok):
        file = bs.BeautifulSoup(myResponse.text, "lxml")
        tag = file.find('response')
        str_name = tag.findAll(['str', 'arr'])
        active = False
        
        for row in str_name:
            if "entrez_id" == row.get("name"):
                entrez_id = row.text
                
            # Check if protein produce from gene
            if "locus_type" == row.get("name"):                
                if row.text == "gene with protein product":
                    active = True
                    continue    
            if  active == True and "uniprot_ids" == row.get("name"):
                uniprot_ids = row.text
                active = False
            
            if "gene_group" == row.get("name"):
                gene_group = row.text
                
            if "gene_group_id" == row.get("name"):
                gene_group_id = row.text
        
        gene_group = str(gene_group).strip()
        gene_group_id = str(gene_group_id).strip()
        
        if "\n" in gene_group:
            gene_group_split = gene_group.split("\n")
            gene_group = gene_group_split[0]
            
        if "\n" in gene_group_id:
            gene_group_id_split = gene_group_id.split("\n")
            gene_group_id = gene_group_id_split[0]
            
                
    return(entrez_id, uniprot_ids, gene_group, gene_group_id)

In [ ]:
def ncbi_id(entrez_id):
    name = "NULL"
    desc = "NULL"

    ncbi_url = "https://www.ncbi.nlm.nih.gov/gene/{}".format(entrez_id)
    page = requests.get(ncbi_url)
    soup = bs.BeautifulSoup(page.text, "html.parser")
    results = soup.find(id="summaryDl")
    res = results.find_all(['dd', 'dt'])

    active_summary = False
    active_name = False
    for row in res:
        if "Summary" in row.text:
            active_summary = True
        elif active_summary == True:
            desc = row.text
            active_summary = False

        if "Full Name" in row.text:
            active_name = True
        elif active_name == True:
            name = row.text.replace("provided by HGNC", "")
            active_name = False
    #print(name, desc, ncbi_url)
    return(name, desc, ncbi_url)


In [ ]:
gene_wprot_dict = dict()
gene_group_list = list() 

gene_symbol_group_list = list()
gene_group_id_list = list()
gene_group_map_dict = dict()

try:
    gene_wprot_dict = pickle.load(open("gene_wprot_dict.pkl", "rb"))
    gene_group_map_dict = pickle.load(open("gene_group_map_dict.pkl", "rb"))

except:
    gene_list = list()
    for gene in gene_dict.keys():
        gene_list.append(gene)
    
    #pprint.pprint(gene_list)
    
    # entrez_id 
    entrez_id_list = list()
    gene_wprot_dict = dict()

    # Genes not in current Harmonizome DB
    leftover_gene_set = set(gene_symbol_list) - set(gene_list)
    
    for gene in leftover_gene_set:
        id_index = int(id_index) + 1

        try:
            url = "https://rest.genenames.org/fetch/symbol/{}".format(gene)
            entrez_id, uniprot_ids, gene_group, gene_group_id = get_url(url)
            gene_group_list.append(str(gene_group).strip()) 
            gene_group_id_list.append(str(gene_group_id).strip())
            gene_symbol_group_list.append(gene)
            
            
            gene = gene.strip()
            uniprot_ids = uniprot_ids.strip()

            if uniprot_ids != "":
                gene_wprot_dict[id_index] = uniprot_ids
            
            print(gene, "---")
            name, desc, ncbi_url = ncbi_id(entrez_id)
            print(gene, "***")
            gene_dict[gene] = [id_index, gene, entrez_id, name, desc, ncbi_url, "NULL", "NULL"]
        except:
            print("---",gene)
            gene_dict[gene] = [id_index, gene, "NULL", "NULL", "NULL", "NULL", "NULL", "NULL"]

    for index in range(0, len(gene_group_id_list)):
        gene_group_map_dict[gene_symbol_group_list[index]] = [gene_group_id_list[index], gene_group_list[index]]       
    
    pprint.pprint(gene_group_map_dict)
    
    pickle.dump(gene_dict, open("gene_dict.pkl", "wb"))
    pickle.dump(gene_wprot_dict, open("gene_wprot_dict.pkl", "wb"))
    pickle.dump(gene_group_map_dict, open("gene_group_map_dict.pkl", "wb"))

In [ ]:
TableOutput('gene', gene_dict, schema_dict['gene'])

In [ ]:
genesymbol_to_geneid_dict = dict()

for key, value in gene_dict.items():
    genesymbol_to_geneid_dict[value[1]] = value[0]

In [ ]:
fillTable(gene_dict, open_sql)

### Write leftover genes in dataset not in current Harmonizome

In [ ]:
def extractHarAll(begin, end):
    # Create Gene Dictionary
    gen_dict = dict() # Key: ID, Value: List of field
    
    # Extract information from Harmonizome.sql
    begin = begin
    end = end + 1

    length = end - begin
    id_index = 0
    with open('harmonizome.sql', 'r') as f:
        count = 0
        for line in itertools.islice(f, begin-1, end):
            line = str(line)
            if count == length:
                print(line)
                break
            count += 1
            quote = False
            slash = False

            row = line.rstrip()
            if row[0] == '(':
                final_str = ''
                for i in row:
                    if i == "'" and quote == False:
                        quote = True
                    elif slash == True:
                        slash = False
                    elif "\\" == i and quote == True:
                        slash = True
                    elif i == "'" and quote == True:
                        quote = False
                    elif quote == True:
                        if "," in i:
                            final_str += '*'
                        else:
                            final_str += i
                    else:
                        final_str += i
                final_str = final_str.replace("'", "")
                final_str = final_str[1:-2]
                split_list = final_str.split(',')
                gene_syn = split_list[0]

                row_list = list()

                for value in split_list:
                    if '*' in value:
                        row_list.append(value.replace("*", ","))
                    else:
                        if value == "":
                            row_list.append("NULL")
                        else:
                            row_list.append(value)

                gen_dict[gene_syn] = row_list

                id_index = split_list[0]
    return(gen_dict, id_index)

# ----------------------------------------------------------
### Write Gene Syn Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('gene_synonym', open_sql)

### Parse Homo Sapiens Gene Info

In [ ]:
open_sapiens = open("Homo_sapiens.gene_info.tsv", "r")
open_sapiens.readline()

gene_syn_dict = dict()

homo_gene_syn_dict = dict()
for row in open_sapiens:
    split_list = row.split("\t")
    gene_syn_list = split_list[4].split("|")
    
    final_gene_syn_list = list()
    for gene in gene_syn_list:
        if "-" == gene:
            final_gene_syn_list.append("NULL")
        else:
            final_gene_syn_list.append(gene)
        
    gene_symbol = split_list[2]
    homo_gene_syn_dict[gene_symbol] = final_gene_syn_list

dup_gene_list = list()
count = 0
for key, value in homo_gene_syn_dict.items():
    gene_symbol = key
    if gene_symbol in genesymbol_to_geneid_dict.keys(): 
        for gene in value:
            if gene == 'NULL':
                continue
            if gene.upper() not in dup_gene_list:      
                count += 1
                gene_syn_dict[count] = [count, gene, genesymbol_to_geneid_dict[gene_symbol]]
                dup_gene_list.append(gene.upper())

### Insert values Gene Syn into master SQL

In [ ]:
beginInsert('gene_synonym')

In [ ]:
fillTable(gene_syn_dict, open_sql)

In [ ]:
TableOutput('gene_synonym', gene_syn_dict, schema_dict['gene_synonym'])

# ----------------------------------------------------------
### Write Protein Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('protein', open_sql)

### Insert values Protein into master SQL

In [ ]:
# Collect Gene Id from gene_dict
gene_id_list = list()
for key, value in gene_dict.items():
    gene_id_list.append(value[0])

In [ ]:
temp_protein_dict, index_idx = extractHarCom(72901445, 72921124, gene_id_list, 6)

In [ ]:
protein_dict = dict()
for key, value in temp_protein_dict.items():
    protein_dict[value[1]] = value

In [ ]:
pprint.pprint(protein_dict)

### Add Protein not in current db

In [ ]:
index_idx = int(index_idx)
try:
    protein_dict = pickle.load(open("protein_dict.pkl", "rb"))

except:
    for key, value in gene_wprot_dict.items():
        gene = key
        uniprot_id = value

        protein_name = ""
        prot_desc = ""
        alternative_name = ""
        url = "https://www.uniprot.org/uniprot/{}".format(value)
        myResponse = requests.get(url)

        if (myResponse.ok):
            soup = bs.BeautifulSoup(myResponse.text, "html.parser")
            meta_res = soup.find_all("meta")
            for row in meta_res:
                if row.get("name") == "description":
                    prot_desc = row.get('content')
                    break

            span_res = soup.find_all("span")
            for row in span_res:
                if row.get("property") == "alternateName":
                    alternative_name = row.text    
                    alternative_name = alternative_name.replace("(", "").replace(")", "")
                    break

            h1_res = soup.find_all("h1")
            for row in h1_res:
                if row.get("property") == "name":
                    protein_name = row.text
                    break
        
        if alternative_name == '':
            continue
        index_idx += 1
        protein_dict[alternative_name] = [int(index_idx) + 1, alternative_name, uniprot_id, protein_name, prot_desc, url, key]
    pickle.dump(protein_dict, open("protein_dict.pkl", "wb" ))

In [ ]:
pprint.pprint(protein_dict)

In [ ]:
TableOutput('protein', protein_dict, schema_dict['protein'])

In [ ]:
beginInsert('protein')
fillTable(protein_dict, open_sql)

# ----------------------------------------------------------
### Write Dataset To Publication Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('datasets_to_publications', open_sql)

### Insert values Dataset To Publication into master SQL

In [ ]:
dataset_id_list = list()
for dataset_id in dataset_dict.keys():
    dataset_id_list.append(dataset_id)

In [ ]:
datasets_to_publications_dict, index_idx = extractHarCom(71901433, 71901603, dataset_id_list, 1)

In [ ]:
beginInsert('datasets_to_publications')
fillTable(datasets_to_publications_dict, open_sql)

# ----------------------------------------------------------
### Write Geneset Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('gene_set', open_sql)

### Insert values Geneset into master SQL

In [ ]:
# Read Geneset List TSV (processed data)
open_geneset = open("attribute_list.tsv", "r")

header = open_geneset.readline()

geneset_list = list()
for row in open_geneset:
    split_genelist = row.split("\t")
    attribute = split_genelist[0]
    attribute_id = split_genelist[1]

    geneset_list.append(attribute)

open_geneset.close()

In [ ]:
def extractHarComDup(begin, end, compare_list, field_one, field_two, dataset_fk):
    # Create Gene Dictionary
    gen_dict = dict() # Key: ID, Value: List of field
    
    # Extract information from Harmonizome.sql
    begin = begin
    end = end + 1

    length = end - begin
    id_index = 0
    with open('harmonizome.sql', 'r') as f:
        count = 0
        for line in itertools.islice(f, begin-1, end):
            line = str(line)
            if count == length:
                print(line)
                break
            count += 1
            quote = False
            slash = False

            row = line.rstrip()
            if row[0] == '(':
                final_str = ''
                for i in row:
                    if i == "'" and quote == False:
                        quote = True
                    elif slash == True:
                        slash = False
                    elif "\\" == i and quote == True:
                        slash = True
                    elif i == "'" and quote == True:
                        quote = False
                    elif quote == True:
                        if "," in i:
                            final_str += '*'
                        else:
                            final_str += i
                    else:
                        final_str += i
                final_str = final_str.replace("'", "")
                final_str = final_str[1:-2]
                split_list = final_str.split(',')
                
                geneset_name = split_list[field_one]
                dataset_id = split_list[field_two]
                
                identifier = split_list[0]
                
                if geneset_name in compare_list and dataset_id == dataset_fk:
                    row_list = list()

                    for value in split_list:
                        if '*' in value:
                            row_list.append(value.replace("*", ","))
                        else:
                            row_list.append(value)

                    gen_dict[identifier] = row_list

                id_index = split_list[0]
    return(gen_dict, id_index)

In [ ]:
# begin, end, compare_list, field_one, field_two, dataset_fk
geneset_dict, index_idx = extractHarComDup(71963633, 72296024, geneset_list, 1, 5, "1")

In [ ]:
geneset_to_genesetid_dict = dict()

for key, value in geneset_dict.items():
    geneset_to_genesetid_dict[value[1]] = value[0]

In [ ]:
geneset_id_list = list()
for geneset_id, value in geneset_dict.items():
    geneset_id_list.append(geneset_id)

In [ ]:
beginInsert('gene_set')
fillTable(geneset_dict, open_sql)

In [ ]:
TableOutput('gene_set', geneset_dict, schema_dict['gene_set'])

# ----------------------------------------------------------
### Write Attribute Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('attribute', open_sql)

### Insert values Attribute into master SQL

In [ ]:
attribute_list = list()
for key, value in geneset_dict.items():
    attribute_list.append(value[7])

attribute_dict, index_idx = extractHarCom(71601358, 71896880, attribute_list, 0)

In [ ]:
beginInsert('attribute')
fillTable(attribute_dict, open_sql)

In [ ]:
TableOutput('attribute', attribute_dict, schema_dict['attribute'])

# ----------------------------------------------------------
### Write Associations Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('association', open_sql)

### Insert values Associations into master SQL

try:
    association_dict = pickle.load(open("association_dict.pkl", "rb"))

except:
    
    geneset_id_list = list()
    gene_id_list = list()
    
    for key, value in gene_dict.items():
        gene_id_list.append(value[0])
        
    for key, value in geneset_dict.items():
        geneset_id_list.append(value[0])
    
    pprint.pprint(gene_id_list)
    pprint.pprint(geneset_id_list)
    
    #association_dict, index_idx = extractHarCom(50, 71601328, gene_id_list, 1)
    
    association_dict = dict() # Key: ID, Value: List of field
    
    # Extract information from Harmonizome.sql
    begin = 50
    end = 71601328

    length = end - begin
    id_index = 0
    
    left_over_geneset_id_list = list()
    left_over_gene_id_list = list()
    with open('harmonizome.sql', 'r') as f:
        count = 0
        for line in itertools.islice(f, begin-1, end):
            if count == length:
                print(line)
                break
            count += 1
            quote = False
            slash = False

            row = line.rstrip()
            if row[0] == '(':
                final_str = ''
                for i in row:
                    if i == "'" and quote == False:
                        quote = True
                    elif slash == True:
                        slash = False
                    elif "\\" == i and quote == True:
                        slash = True
                    elif i == "'" and quote == True:
                        quote = False
                    elif quote == True:
                        if "," in i:
                            final_str += '*'
                        else:
                            final_str += i
                    else:
                        final_str += i
                final_str = final_str.replace("'", "")
                final_str = final_str[1:-2]
                split_list = final_str.split(',')
                
                index = split_list[0]
                gene = split_list[1]
                gene_set = split_list[2]
                
                if gene in gene_id_list and gene_set in geneset_id_list:
                    row_list = list()

                    for value in split_list:
                        if '*' in value:
                            value = value.replace("*", ",")
                        
                        if value.isnumeric():
                            row_list.append(int(value))
                        else:
                            row_list.append(value)

                    association_dict[index] = row_list
                else:
                    left_over_geneset_id_list.append(gene_set)
                    left_over_gene_id_list.append(gene)
                    
                id_index = split_list[0]
            
    print(id_index)

    #association_dict, index_idx = extractHarAll(50, 71601328)
    pickle.dump(association_dict, open("association_dict.pkl", "wb" ))

In [ ]:
#association_index = 1290000029563
f = h5.File("gene_attribute_matrix_standardized.h5", "r")

datasetNames = [n for n in f.keys()]
data = f['data']['matrix']

meta_colid = f['meta']['colid']
meta_index = f['meta']['index']

geneset_list = list()
for geneset in meta_colid:
    geneset_list.append(geneset.decode('UTF-8'))

gene_list = list()
for gene in meta_index:
    gene_list.append(gene.decode('UTF-8'))

association_dict = dict()
association_index = 1000000000

# geneset_to_genesetid_dict
# genesymbol_to_geneid_dict 

def threshholdValue(value):
    if value < 0:
        return -1
    else: 
        return 1

temp_association_dict = dict()

for index_gene, geneset_row in enumerate(data):
    gene = gene_list[index_gene] 
    
    if gene in genesymbol_to_geneid_dict.keys():
        gene_id = genesymbol_to_geneid_dict[gene]
    else:
        continue
        #gene_id = "NULL"
    
    for index_geneset, gene_value in enumerate(geneset_row):
        geneset = geneset_list[index_geneset]
        geneset_id = geneset_to_genesetid_dict[geneset]
        
        if geneset_id == "NULL":
            continue
        
        thvalue = threshholdValue(gene_value)
        
        if thvalue == 1:
            if gene_value < 1:
                continue
        elif thvalue == -1:
            if gene_value > -1:
                continue
        
        association_index += 1
        temp_association_dict[association_index] = [association_index, gene, geneset_id, "NULL", "NULL", round(float(gene_value), 5), thvalue]


In [ ]:
temp_two_association_dict = dict()
association_index = 1000000000

for key, value in temp_association_dict.items():
    gene = value[1]
    geneset_id = value[2]
    gene_value = value[5]
    
    if geneset_id not in temp_two_association_dict.keys():
        temp_two_association_dict[geneset_id] = list()
    else:
        temp_two_association_dict[geneset_id].append([gene, gene_value])

print(len(temp_two_association_dict))
association_dict = dict()
for key, value in temp_two_association_dict.items():
    geneset_id = key
    genes_in_geneset_list = temp_two_association_dict[key]
    genes_in_geneset_list.sort(key = lambda x: x[1])
    
    for pair1, pair2 in genes_in_geneset_list[-30:]:
        genesymbol = pair1
        gene_value = pair2
        
        thvalue = threshholdValue(gene_value)
        
        association_index += 1
        association_dict[association_index] = [association_index, genesymbol_to_geneid_dict[genesymbol], geneset_id, "NULL", "NULL", gene_value, thvalue]
    
    for pair3, pair4 in genes_in_geneset_list[:30]:
        genesymbol = pair3
        gene_value = pair4
        
        thvalue = threshholdValue(gene_value)
        
        association_index += 1
        association_dict[association_index] = [association_index, genesymbol_to_geneid_dict[genesymbol], geneset_id, "NULL", "NULL", gene_value, thvalue]

 

In [ ]:
pprint.pprint(association_dict)

In [ ]:
def fillTableAssoc(resource_dict, open_sql):
    index = 0
    for key, value in resource_dict.items():
        if index == len(resource_dict.keys())-1:
            open_sql.write('\n(')
            for count, entry in enumerate(value):
                entry = str(entry)
                if count == len(value)-1:
                    entry = entry.rstrip()
                    if entry == " " or entry == "":
                        open_sql.write("{});".format("NULL"))
                    elif entry.isnumeric() or isinstance(entry, float):
                        open_sql.write("{});".format(entry))
                    elif "NULL" == entry or "Null" == entry:
                        open_sql.write("{});".format(entry))
                    else:
                        open_sql.write('{});'.format(entry))
                else:
                    if entry == " " or entry == "":
                        open_sql.write("{},".format("NULL"))
                    elif entry.isnumeric() or isinstance(entry, float):
                        open_sql.write("{},".format(entry))
                    elif "NULL" == entry:
                        open_sql.write("{},".format(entry))
                    else:
                        open_sql.write('{},'.format(entry))
            break
            
        open_sql.write('\n(')   
        for count, entry in enumerate(value):
            entry = str(entry)
            if count == len(value)-1:
                entry = entry.rstrip()
                if entry == " " or entry == "":
                    open_sql.write("{}),".format("NULL"))
                elif entry.isnumeric() or isinstance(entry, float):
                    open_sql.write("{}),".format(entry))
                elif "NULL" == entry or "Null" == entry:
                    open_sql.write("{}),".format(entry))
                else:
                    open_sql.write('{}),'.format(entry))
            else:
                if entry == " " or entry == "":
                    open_sql.write("{},".format("NULL"))
                elif entry.isnumeric() or isinstance(entry, float):
                    open_sql.write("{},".format(entry))
                elif "NULL" == entry:
                    open_sql.write("{},".format(entry))
                else:
                    open_sql.write('{},'.format(entry))
        index += 1
        
    open_sql.write("\nUNLOCK TABLES;")

In [ ]:
beginInsert('association')
fillTableAssoc(association_dict, open_sql)
TableOutput('association', association_dict, schema_dict['association'])

# ----------------------------------------------------------
### Write Attribute Type Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('attribute_type', open_sql)

### Insert values Attribute Type into master SQL

In [ ]:
open_attribute_type = open("attribute_type.tsv", "r")

# Read Header
header = open_attribute_type.readline()
# Read Example
example = open_attribute_type.readline()

# Key = ID, Value = Publications Name
attribute_type_dict = dict()
for row in open_attribute_type:
    split_row =  row.split("\t")
    
    if split_row[1] == '':
        continue
    
    if '\n' in split_row[len(split_row)-1]:
        split_row[len(split_row)-1] = split_row[len(split_row)-1].rstrip()
        
    attribute_type_id = split_row[0]
    
    count = 0
    field_list = list()
    while count < 3:
        if split_row[count] != '':
            field_value = split_row[count]
            if '"' in field_value:
                field_value = field_value.replace('"', '')
            field_list.append(field_value)
        else:
            field_list.append("NULL")
        count += 1
    
    attribute_type_dict[attribute_type_id] = field_list

open_attribute_type.close()  

In [ ]:
beginInsert('attribute_type')
fillTable(attribute_type_dict, open_sql)
TableOutput('attribute_type', attribute_type_dict, schema_dict['attribute_type'])

# ----------------------------------------------------------
### Write Attribute Group Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('attribute_group', open_sql)

### Insert values Attribute Group into master SQL

In [ ]:
open_attribute_group = open("attribute_group.tsv", "r")

# Read Header
header = open_attribute_group.readline()
# Read Example
example = open_attribute_group.readline()

# Key = ID, Value = Publications Name
attribute_group_dict = dict()
for row in open_attribute_group:
    split_row =  row.split("\t")
    
    if split_row[1] == '':
        continue
    
    if '\n' in split_row[len(split_row)-1]:
        split_row[len(split_row)-1] = split_row[len(split_row)-1].rstrip()
        
    attribute_group_id = split_row[0]
    
    count = 0
    field_list = list()
    while count < 3:
        if split_row[count] != '':
            field_value = split_row[count]
            if '"' in field_value:
                field_value = field_value.replace('"', '')
            field_list.append(field_value)
        else:
            field_list.append("NULL")
        count += 1
    
    attribute_group_dict[attribute_group_id] = field_list

open_attribute_group.close()

In [ ]:
beginInsert('attribute_group')
fillTable(attribute_group_dict, open_sql)
TableOutput('attribute_group', attribute_group_dict, schema_dict['attribute_group'])

# ----------------------------------------------------------
### Write Dataset Group Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('dataset_group', open_sql)

### Insert values Dataset Group into master SQL

In [ ]:
open_dataset_group = open("dataset_group.tsv", "r")

# Read Header
header = open_dataset_group.readline()
# Read Example
example = open_dataset_group.readline()

# Key = ID, Value = Publications Name
dataset_group_dict = dict()
for row in open_dataset_group:
    split_row =  row.split("\t")
    
    if split_row[1] == '':
        continue
    
    if '\n' in split_row[len(split_row)-1]:
        split_row[len(split_row)-1] = split_row[len(split_row)-1].rstrip()
        
    dataset_group_id = split_row[0]
    
    count = 0
    field_list = list()
    while count < 3:
        if split_row[count] != '':
            field_value = split_row[count]
            if '"' in field_value:
                field_value = field_value.replace('"', '')
            field_list.append(field_value)
        else:
            field_list.append("NULL")
        count += 1
    
    dataset_group_dict[dataset_group_id] = field_list

open_dataset_group.close()

In [ ]:
beginInsert('dataset_group')
fillTable(dataset_group_dict, open_sql)
TableOutput('dataset_group', dataset_group_dict, schema_dict['dataset_group'])

# ----------------------------------------------------------
### Write dataset_pair_visualization Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('dataset_pair_visualization', open_sql)

### Insert values dataset_pair_visualization into master SQL

In [ ]:
dataset_id_list = list()
for key, item in dataset_dict.items():
    dataset_id_list.append(item[0])

In [ ]:
def extractHarComTwo(begin, end, compare_list, field_one, field_two):
    # Create Gene Dictionary
    gen_dict = dict() # Key: ID, Value: List of field
    
    # Extract information from Harmonizome.sql
    begin = begin
    end = end + 1

    length = end - begin
    id_index = 0
    with open('harmonizome.sql', 'r') as f:
        count = 0
        for line in itertools.islice(f, begin-1, end):
            line = str(line)
            if count == length:
                print(line)
                break
            count += 1
            quote = False
            slash = False

            row = line.rstrip()
            if row[0] == '(':
                final_str = ''
                for i in row:
                    if i == "'" and quote == False:
                        quote = True
                    elif slash == True:
                        slash = False
                    elif "\\" == i and quote == True:
                        slash = True
                    elif i == "'" and quote == True:
                        quote = False
                    elif quote == True:
                        if "," in i:
                            final_str += '*'
                        else:
                            final_str += i
                    else:
                        final_str += i
                final_str = final_str.replace("'", "")
                final_str = final_str[1:-2]
                split_list = final_str.split(',')
                
                dataset_one = split_list[field_one]
                dataset_two = split_list[field_two]
                
                identifier = split_list[0]

                if dataset_one in compare_list and dataset_two in compare_list:
                    row_list = list()

                    for value in split_list:
                        if '*' in value:
                            row_list.append(value.replace("*", ","))
                        else:
                            row_list.append(value)

                    gen_dict[identifier] = row_list

                id_index = split_list[0]
    return(gen_dict, id_index)

In [ ]:
dataset_pair_visualization_dict, index_idx = extractHarComTwo(71897222, 71900875, dataset_id_list, 1, 2)

In [ ]:
beginInsert('dataset_pair_visualization')
fillTable(dataset_pair_visualization_dict, open_sql)
TableOutput('dataset_pair_visualization', dataset_pair_visualization_dict, schema_dict['dataset_pair_visualization'])

# ----------------------------------------------------------
### Write dataset_visualization Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('dataset_visualization', open_sql)

### Insert values dataset_visualization into master SQL

In [ ]:
dataset_visualization_dict, index_idx = extractHarCom(71901076, 71901403, dataset_id_list, 1)

In [ ]:
beginInsert('dataset_visualization')
fillTable(dataset_visualization_dict, open_sql)
TableOutput('dataset_visualization', dataset_visualization_dict, schema_dict['dataset_visualization'])

# ----------------------------------------------------------
### Write dataset_test Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('dataset_test', open_sql)

### Insert values dataset_test into master SQL

In [ ]:
dataset_test_dict, index_idx = extractHarCom(71900936, 71901047, dataset_id_list, 15)

In [ ]:
beginInsert('dataset_test')
fillTable(dataset_test_dict, open_sql)
TableOutput('dataset_test', dataset_test_dict, schema_dict['dataset_test'])

# ----------------------------------------------------------
### Write devi_report Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('devi_report', open_sql)

### Insert values devi_report into master SQL

In [ ]:
# geneset_list - Attribute names
devi_report_dict, index_idx = extractHarCom(71901630, 71904444, geneset_list, 2)

In [ ]:
beginInsert('devi_report')
fillTable(devi_report_dict, open_sql)
TableOutput('devi_report', devi_report_dict, schema_dict['devi_report'])

# ----------------------------------------------------------
### Write user_search Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('user_search', open_sql)

### Insert values user_search into master SQL

In [ ]:
user_search_dict, index_idx = extractHarAll(72921456, 73266718)

In [ ]:
beginInsert('user_search')
fillTable(user_search_dict, open_sql)

# ----------------------------------------------------------
### Write download Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('download', open_sql)

### Insert values download into master SQL

In [ ]:
download_dict, index_idx = extractHarCom(71904476, 71905659, dataset_id_list, 2)

beginInsert('download')
fillTable(download_dict, open_sql)
TableOutput('download', download_dict, schema_dict['download'])

# ----------------------------------------------------------
### Write download_type Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('download_type', open_sql)

### Insert values download_type into master SQL

In [ ]:
download_type_dict, index_idx = extractHarAll(71906805, 71906822)

beginInsert('download_type')
fillTable(download_type_dict, open_sql)
TableOutput('download_type', download_type_dict, schema_dict['download_type'])

# ----------------------------------------------------------
### Write measurement Schema Table into master SQL


In [ ]:
open_sql.write("\n")
printSchema('measurement', open_sql)

### Insert measurement into master SQL

In [ ]:
measurement_dict, index_idx = extractHarAll(72410301, 72410329)

beginInsert('measurement')
fillTable(measurement_dict, open_sql)
TableOutput('measurement', measurement_dict, schema_dict['measurement'])

# ----------------------------------------------------------
### Write naming_authority Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('naming_authority', open_sql)

### Insert naming_authority into master SQL

In [ ]:
naming_authority_dict, index_idx = extractHarAll(72901318, 72901409)

beginInsert('naming_authority')
fillTable(naming_authority_dict, open_sql)
TableOutput('naming_authority', naming_authority_dict, schema_dict['naming_authority'])

# ----------------------------------------------------------
### Write hgnc_root_family Schema Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('hgnc_root_family', open_sql)

### Insert hgnc_root_family and hgnc_root_families_to_genes into master SQL

In [ ]:
hgnc_root_family_dict, index_idx_hgnc_root = extractHarAll(72382327, 72382635)

In [ ]:
hgnc_root_families_to_genes_dict, index_idx_hgnc_genes = extractHarCom(72364694, 72382298, gene_id_list, 1)

In [ ]:
# Key: Group Name, Value: entry in list form
temp_hgnc_root_family_dict = dict()
for key, value in hgnc_root_family_dict.items():
    temp_hgnc_root_family_dict[value[1]] = value

In [ ]:
pprint.pprint(temp_hgnc_root_family_dict)

In [ ]:
# Key: gene symbol, Value: [gene_fk, gene_symbol]
# gene_group_map_dict = {'H1-6': ['1935', 'H1 histones']}

gene_group_map_dict = pickle.load(open("gene_group_map_dict.pkl", "rb"))

for key, value in gene_group_map_dict.items():
    if value[0] != "" and value[1] != "":
        if value[1] not in temp_hgnc_root_family_dict.keys():
            hgnc_url = "https://www.genenames.org/data/genegroup/#!/group/{}".format(value[0])
            index_idx_hgnc_root = int(index_idx_hgnc_root) + 1
            hgnc_root_family_dict[value[1]] = [index_idx_hgnc_root, value[1], value[0], hgnc_url]

pprint.pprint(hgnc_root_family_dict)

In [ ]:
genegroupname_to_hgnc_root_family_fk_dict = dict()
for key, value in hgnc_root_family_dict.items():
    genegroupname_to_hgnc_root_family_fk_dict[value[1]] = value[0]

In [ ]:
pprint.pprint(genegroupname_to_hgnc_root_family_fk_dict)

In [ ]:
pprint.pprint(gene_group_map_dict)

In [ ]:
genesymbol_to_gene_fk_dict = dict()
for key, value in gene_dict.items():
    genesymbol_to_gene_fk_dict[value[1]] = value[0]
pprint.pprint(genesymbol_to_gene_fk_dict)

In [ ]:
for gene, value in gene_group_map_dict.items():
    if value[0] != "" and value[1] != "":
        if gene in genesymbol_to_gene_fk_dict.items():
            gene_fk = genesymbol_to_gene_fk_dict[gene]  
            if gene_fk not in temp_hgnc_root_family_dict.keys():
                index_idx_hgnc_genes = int(index_idx_hgnc_genes) + 1
                entry_list = [index_idx_hgnc_genes, gene_fk, genegroupname_to_hgnc_root_family_fk_dict[value[1]]]
                hgnc_root_families_to_genes_dict[index_idx_hgnc_genes] = entry_list

In [ ]:
beginInsert('hgnc_root_family')
fillTable(hgnc_root_family_dict, open_sql)
TableOutput('hgnc_root_family', hgnc_root_family_dict, schema_dict['hgnc_root_family'])

open_sql.write("\n")
printSchema('hgnc_root_families_to_genes', open_sql)

beginInsert('hgnc_root_families_to_genes')
fillTable(hgnc_root_families_to_genes_dict, open_sql)
TableOutput('hgnc_root_families_to_genes', hgnc_root_families_to_genes_dict, schema_dict['hgnc_root_families_to_genes'])

# ----------------------------------------------------------
### Write stats Schema Table into master SQL

# ----------------------------------------------------------
### Write stats mp_prediction Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('mp_prediction', open_sql)

### Insert mp_prediction into master SQL

In [ ]:
mp_prediction_dict, index_idx = extractHarCom(72410370, 72901289, gene_id_list, 11)

In [ ]:
beginInsert('mp_prediction')
fillTable(mp_prediction_dict, open_sql)
TableOutput('mp_prediction', mp_prediction_dict, schema_dict['mp_prediction'])

# ----------------------------------------------------------
### Write idg_tdl_class Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('idg_tdl_class', open_sql)

### Insert idg_tdl_class into master SQL

In [ ]:
idg_tdl_class_dict, index_idx = extractHarAll(72384605, 72384610)

In [ ]:
beginInsert('idg_tdl_class')
fillTable(idg_tdl_class_dict, open_sql)
TableOutput('idg_tdl_class', idg_tdl_class_dict, schema_dict['idg_tdl_class'])

# ----------------------------------------------------------
### Write idg_family Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('idg_family', open_sql)

### Insert idg_family into master SQL

In [ ]:
idg_family_dict, index_idx = extractHarAll(72384576, 72384579)

In [ ]:
beginInsert('idg_family')
fillTable(idg_family_dict, open_sql)
TableOutput('idg_family', idg_family_dict, schema_dict['idg_family'])

# ----------------------------------------------------------
### Write ks_prediction Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('ks_prediction', open_sql)

### Insert ks_prediction into master SQL

In [ ]:
ks_prediction_dict, index_idx = extractHarAll(72384653, 72410275)

In [ ]:
beginInsert('ks_prediction')
fillTable(ks_prediction_dict, open_sql)
TableOutput('ks_prediction', ks_prediction_dict, schema_dict['ks_prediction'])

# ----------------------------------------------------------
### Write ic_prediction Table into master SQL

In [ ]:
open_sql.write("\n")
printSchema('ic_prediction', open_sql)

### Insert ic_prediction into master SQL

In [ ]:
ic_prediction_dict, index_idx = extractHarCom(72382682, 72384550, gene_id_list, 21)

In [ ]:
beginInsert('ic_prediction')
fillTable(ic_prediction_dict, open_sql)
TableOutput('ic_prediction', ic_prediction_dict, schema_dict['ic_prediction'])

# ----------------------------------------------------------

In [ ]:
open_sql.close()